In [1]:
# import the libraries

import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import statsmodels.api as sm
import numpy as np
from pandas_datareader.famafrench import get_available_datasets

In [2]:
get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
# Read in three factors (market, size, book to market) from Professor French's website as a dataframe
ff = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start = '1991-1-1')[0]

ff.head()

/var/folders/rx/wfm78crj6bg1zsfg6pssdc6r0000gn/T/ipykernel_78494/1019251418.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start = '1991-1-1')[0]


,Mkt-RF,SMB,HML,RF
Date,,,,
1991-01-02,-0.95,0.64,0.82,0.023
1991-01-03,-1.25,0.28,1.17,0.023
1991-01-04,-0.24,0.12,0.42,0.023
1991-01-07,-1.72,0.32,0.23,0.023
1991-01-08,-0.29,-0.36,-0.01,0.023


In [4]:
# Read in the data of Intel, AMD, and NVIDIA

yf.pdr_override()

tickers = ['INTL', 'AMD', 'NVDA']

mydata = pd.DataFrame()

for t in tickers:
    mydata[t] = pdr.get_data_yahoo(t)['Adj Close']

mydata.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,INTL,AMD,NVDA
Date,,,
2022-12-02,19.344280,74.980003,168.677231
2022-12-05,19.087631,73.620003,166.018555
2022-12-06,19.075182,70.269997,159.791595
2022-12-07,19.020597,70.139999,161.120941
2022-12-08,19.179567,70.470001,171.605804


In [5]:
# Calculate the simple daily returns using Adj. Close

mydata_returns = (mydata / mydata.shift(1) - 1) * 100 

mydata_returns.head()

,INTL,AMD,NVDA
Date,,,
2022-12-02,NaN,NaN,NaN
2022-12-05,-1.326744,-1.813818,-1.576191
2022-12-06,-0.065222,-4.550402,-3.750761
2022-12-07,-0.286155,-0.184997,0.831925
2022-12-08,0.835778,0.470490,6.507449


In [6]:
#Merge the factors with stock returns

all = pd.merge(mydata_returns, ff, left_index = True, right_index = True)
all.head()

,INTL,AMD,NVDA,Mkt-RF,SMB,HML,RF
Date,,,,,,,
2022-12-02,NaN,NaN,NaN,-0.08,0.92,-0.55,0.016
2022-12-05,-1.326744,-1.813818,-1.576191,-2.02,-0.59,-0.51,0.016
2022-12-06,-0.065222,-4.550402,-3.750761,-1.51,-0.24,1.16,0.016
2022-12-07,-0.286155,-0.184997,0.831925,-0.24,-0.13,-0.34,0.016
2022-12-08,0.835778,0.470490,6.507449,0.77,0.20,-1.22,0.016


In [7]:
all = all.dropna()
all.head()

,INTL,AMD,NVDA,Mkt-RF,SMB,HML,RF
Date,,,,,,,
2022-12-05,-1.326744,-1.813818,-1.576191,-2.02,-0.59,-0.51,0.016
2022-12-06,-0.065222,-4.550402,-3.750761,-1.51,-0.24,1.16,0.016
2022-12-07,-0.286155,-0.184997,0.831925,-0.24,-0.13,-0.34,0.016
2022-12-08,0.835778,0.470490,6.507449,0.77,0.20,-1.22,0.016
2022-12-09,-0.384472,-2.667809,-0.978521,-0.80,-0.55,0.36,0.016


In [8]:
# Calculate excess stocks returns (stock return - rf)

all['INTL - RF'] = all['INTL'] - all['RF']
all['AMD - RF'] = all['AMD'] - all['RF']
all['NVDA - RF'] = all['NVDA'] - all['RF']
all.head()

,INTL,AMD,NVDA,Mkt-RF,SMB,HML,RF,INTL - RF,AMD - RF,NVDA - RF
Date,,,,,,,,,,
2022-12-05,-1.326744,-1.813818,-1.576191,-2.02,-0.59,-0.51,0.016,-1.342744,-1.829818,-1.592191
2022-12-06,-0.065222,-4.550402,-3.750761,-1.51,-0.24,1.16,0.016,-0.081222,-4.566402,-3.766761
2022-12-07,-0.286155,-0.184997,0.831925,-0.24,-0.13,-0.34,0.016,-0.302155,-0.200997,0.815925
2022-12-08,0.835778,0.470490,6.507449,0.77,0.20,-1.22,0.016,0.819778,0.454490,6.491449
2022-12-09,-0.384472,-2.667809,-0.978521,-0.80,-0.55,0.36,0.016,-0.400472,-2.683809,-0.994521


In [9]:
# Run a CAPM model to evaluate performance of the three stocks

X = all['Mkt-RF']
X1 = sm.add_constant(X)

reg1 = sm.OLS(all['INTL - RF'], X1).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INTL - RF   R-squared:                       0.581
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     398.7
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           2.78e-56
Time:                        16:12:12   Log-Likelihood:                -259.87
No. Observations:                 290   AIC:                             523.7
Df Residuals:                     288   BIC:                             531.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0131      0.035     -0.375      0.708      -0.082       0.056
Mkt-RF         0.7898      0.040     19.968      0.000       0.712       0.868
==============================================================================
Omnibus:                        9.650   Durbin-Watson:                   2.070
Prob(Omnibus):                  0.008   Jarque-Bera (JB):               13.311
Skew:                          -0.250   Prob(JB):                      0.00129
Kurtosis:                       3.923   Cond. No.                         1.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
reg2 = sm.OLS(all['AMD - RF'], X1).fit()
reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               AMD - RF   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     131.5
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           2.50e-25
Time:                        16:12:28   Log-Likelihood:                -674.33
No. Observations:                 290   AIC:                             1353.
Df Residuals:                     288   BIC:                             1360.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2111      0.146      1.445      0.149      -0.076       0.499
Mkt-RF         1.8939      0.165     11.468      0.000       1.569       2.219
==============================================================================
Omnibus:                       45.761   Durbin-Watson:                   2.106
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.165
Skew:                           0.781   Prob(JB):                     6.53e-23
Kurtosis:                       5.452   Cond. No.                         1.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
reg3 = sm.OLS(all['NVDA - RF'], X1).fit()
reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              NVDA - RF   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     139.2
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           1.79e-26
Time:                        16:12:43   Log-Likelihood:                -676.08
No. Observations:                 290   AIC:                             1356.
Df Residuals:                     288   BIC:                             1364.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3773      0.147      2.567      0.011       0.088       0.667
Mkt-RF         1.9604      0.166     11.799      0.000       1.633       2.287
==============================================================================
Omnibus:                      241.921   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7845.936
Skew:                           3.069   Prob(JB):                         0.00
Kurtosis:                      27.732   Cond. No.                         1.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Run a Fama-French three-factor model 
# to evaluate performance of the three funds

X = all[['Mkt-RF', 'SMB', 'HML']]
X1 = sm.add_constant(X)

reg1 = sm.OLS(all['INTL - RF'], X1).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INTL - RF   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     151.8
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           7.31e-59
Time:                        16:13:36   Log-Likelihood:                -247.74
No. Observations:                 290   AIC:                             503.5
Df Residuals:                     286   BIC:                             518.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0028      0.034     -0.084      0.933      -0.069       0.064
Mkt-RF         0.7778      0.042     18.684      0.000       0.696       0.860
SMB            0.1005      0.054      1.845      0.066      -0.007       0.208
HML            0.1939      0.046      4.254      0.000       0.104       0.284
==============================================================================
Omnibus:                        7.239   Durbin-Watson:                   2.117
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                7.495
Skew:                          -0.298   Prob(JB):                       0.0236
Kurtosis:                       3.515   Cond. No.                         1.80
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
reg2 = sm.OLS(all['AMD - RF'], X1).fit()
reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               AMD - RF   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     60.71
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           2.12e-30
Time:                        16:13:52   Log-Likelihood:                -657.42
No. Observations:                 290   AIC:                             1323.
Df Residuals:                     286   BIC:                             1338.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1684      0.139      1.215      0.226      -0.104       0.441
Mkt-RF         1.8705      0.171     10.941      0.000       1.534       2.207
SMB           -0.2355      0.224     -1.053      0.293      -0.676       0.205
HML           -1.0454      0.187     -5.585      0.000      -1.414      -0.677
==============================================================================
Omnibus:                       42.277   Durbin-Watson:                   2.108
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.190
Skew:                           0.696   Prob(JB):                     6.45e-23
Kurtosis:                       5.554   Cond. No.                         1.80
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
reg3 = sm.OLS(all['NVDA - RF'], X1).fit()
reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              NVDA - RF   R-squared:                       0.424
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     70.06
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           5.36e-34
Time:                        16:14:12   Log-Likelihood:                -653.37
No. Observations:                 290   AIC:                             1315.
Df Residuals:                     286   BIC:                             1329.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3210      0.137      2.348      0.020       0.052       0.590
Mkt-RF         2.0075      0.169     11.907      0.000       1.676       2.339
SMB           -0.5037      0.221     -2.284      0.023      -0.938      -0.070
HML           -1.1237      0.185     -6.087      0.000      -1.487      -0.760
==============================================================================
Omnibus:                      245.439   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8678.316
Skew:                           3.104   Prob(JB):                         0.00
Kurtosis:                      29.071   Cond. No.                         1.80
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
sec_beta = pd.DataFrame(np.nan, index = tickers, columns = ['const','Mkt-RF','SMB','HML'])

for t in tickers:
    X = all[['Mkt-RF', 'SMB', 'HML']]
    X1 = sm.add_constant(X)
    Y = all[t] - all['RF']
    reg = sm.OLS(Y, X1).fit()
    sec_beta.loc[t, :] = reg.params
sec_beta

,const,Mkt-RF,SMB,HML
INTL,-0.002822,0.777809,0.100456,0.193925
AMD,0.168414,1.870513,-0.235531,-1.045432
NVDA,0.320988,2.007452,-0.503683,-1.123683
